In [145]:
import urllib.request
import bs4
import pandas as pd
import numpy as np
import pickle
import re
from tqdm.notebook import tqdm

def get_df(url):
    ad_url = 'https://www.letsrun.com/shoes/'
    titles = []
    posts = []
    original_post = []
    date = []
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers = {'User-Agent':user_agent,} 
    request = urllib.request.Request(url,None,headers)
    response = urllib.request.urlopen(request)
    webpage = response.read()
    soup = bs4.BeautifulSoup(webpage, 'html.parser')
    body = soup.find_all("ul", class_="forum-index-thread-list")
    entries = body[0].find_all('li', class_='thread')
    for i in entries:
        container = i.a
        if container is not None:
            post_url = container['href']
            if post_url != ad_url:
                title = container.renderContents().decode('utf-8').strip()
                post = i.findAll('span', class_='post-count')[0].text
                titles.append(title)
                posts.append(post.strip(" posts"))
                #request = urllib.request.Request(post_url,None,headers)
                #response = urllib.request.urlopen(request)
                #webpage = response.read()
                #post_soup = bs4.BeautifulSoup(webpage, 'html.parser')
                #print(post_soup.find_all("span", id="intelliTXT"))
                #original_post.append(post_body)
                   
    df = pd.DataFrame(list(zip(titles, posts)), columns =['title', 'posts', ])
    return df


def scrape_pages(num_pages):
    url_base = 'https://www.letsrun.com/forum?page='
    dfs = []
    for i in tqdm(range(num_pages)):
        idx = i + 1
        url = url_base + str(idx)
        dfs.append(get_df(url))    
    df_main = pd.concat(dfs).reset_index()
    df_main.drop(columns=['index'], inplace=True)
    return df_main

In [ ]:
df = scrape_pages(133)
df.to_csv('letsrun_title_data.csv', index=False)

In [149]:
df

,title,posts
0,AMAZON HAS HAD A BLACK LIVES BANNER UP ON THEI...,31
1,Official LRC President Donald J Trump Thread,98697
2,How does qualifying for major marathons work?,7
3,Can we create a separate politics/current even...,1
4,My church was attacked,31
...,...,...
7495,Runners unite worldwide!,2
7496,L. A. County is 1st in the USA with Universal ...,3
7497,Should law enforcement be totally seperated fr...,10
7498,Why is Product better Green Leaves CBD Oil tha...,1
